In [2]:
# Carga de librerías necesarias
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from random import random

> **Ejercicio 1:** Pruebe ahora a repetir el mismo proceso, pero con los datos de interpolación  
> 
> a) $(a-h,f(a-h))$ y $(a,f(a))$,
> 
> b) $(a-h,f(a-h))$ y $(a+h,f(a+h))$. 
> 
> ¿Qué fórmulas de derivación numérica se obtiene en cada caso?

In [39]:
f = sp.Function('f')                # Crea función simbólica
a,h = sp.symbols('a,h')
x_values = [a-h,a]                  # Intervalo simbólico, lista que contiene variables simbólicas
y_values = [f(x_values[0]),f(x_values[1])]
x = sp.Symbol('x')
p = (y_values[1]-y_values[0])/(x_values[1]-x_values[0])*(x-x_values[0])+y_values[0]
p.subs({x:x_values[0]}),p.subs({x:x_values[1]})
sp.diff(p,x) ## formula de diferencia regresiva

x1_values = [a-h,a+h]    
y1_values = [f(x1_values[0]),f(x1_values[1])]
p1 = (y1_values[1]-y1_values[0])/(x1_values[1]-x1_values[0])*(x-x1_values[0])+y1_values[0]
p1.subs({x:x1_values[0]}),p1.subs({x:x1_values[1]})
sp.diff(p1,x)## formula de diferencia centrada



(-f(a - h) + f(a + h))/(2*h)

> **Ejercicio 2:** Codifica una función para calcular la fórmula de derivación de tipo interpolatorio obtenida al derivar el polinomio interpolador en una lista de nodos que acepte como argumento. Añade un argumento opcional que sea el orden de la derivada a calcular. Comprueba el correcto funcionamiento de esta función con los ejemplos mostrados hasta el momento.

In [40]:
def calcula_formula_interp(x_values,orden):
    f = sp.Function('f') 
    x = sp.Symbol('x')
    x_values =sp.symbols(x_values)
    lk=1   
    p=0
    for k in range(0,len(x_values)):
        lk=1
        for i in range(0,len(x_values)):
            if i != k:
                lk *=(x-x_values[i])/(x_values[k]-x_values[i])
        p+=f(x_values[k])*lk
    for i in range(0,orden):
        p = sp.diff(p,x)
    return p


p = calcula_formula_interp(['x1','x2','x3'],1)
p

(x - x1)*f(x3)/((-x1 + x3)*(-x2 + x3)) + (x - x1)*f(x2)/((-x1 + x2)*(x2 - x3)) + (x - x2)*f(x1)/((x1 - x2)*(x1 - x3)) + (x - x2)*f(x3)/((-x1 + x3)*(-x2 + x3)) + (x - x3)*f(x1)/((x1 - x2)*(x1 - x3)) + (x - x3)*f(x2)/((-x1 + x2)*(x2 - x3))

> **Ejercicio 3**: Repita este procedimiento con la fórmula para la segunda derivada
> $$
> f''(a) \approx \frac{f(a+h)-2f(a)+f(a-h)}{h^2}
> $$
> y compruebe que tiene orden de aproximación 2.

In [42]:
p = calcula_formula_interp(['x1','x2','x3'],2)
p

2*f(x1)/((x1 - x2)*(x1 - x3)) + 2*f(x3)/((-x1 + x3)*(-x2 + x3)) + 2*f(x2)/((-x1 + x2)*(x2 - x3))

> **Ejercicio 4:** Utilice la función $f(x) = \dfrac{e^{\cos(x)}}{x^2+1}$ y el valor $a=1$ y compruebe que ocurre este mismo hecho. Repita el procedimiento utilizando la fórmula de diferencia centrada en lugar de la de diferencia progresiva y comenta las diferencias observadas.



> **Ejercicio 5 (Implementación numérica de las fórmulas de derivación):** Implementa una función que acepte como argumentos una función en versión numérica `f`, un valor `a`, un valor `h`, un entero opcional `n` y un cuarto argumento que sea la fórmula de derivación numérica utilizada. La función deberá calcular la derivada `n`-ésima de la función `f` en el punto `a` utilizando la fórmula y el valor de `h` indicados. Las posibles fórmulas utilizadas deben incluir, al menos: diferencia regresiva, diferencia progresiva, diferencia centrada y diferencia centrada con 5 nodos. Para la función $$f(x) = \sin(\log(x^2+1)),$$ calcula el valor de $f'(-1)$, primero derivando una versión simbólica y obteniendo el valor exacto y después utilizando la función implementada con distintos valores de `h` y distintas fórmulas y comprueba las diferencias en la aproximación en cada caso.

> **Ejercicio 6:** Siguiendo el modelo y el código proporcionado para la fórmula de los rectángulos a la izquierda, codifique las fórmulas compuestas de los rectángulos a la derecha y centrados. Calcule con estas dos fórmulas la aproximación de $\displaystyle\int_0^1 (x^2+1)dx$, el error cometido y represente al menos una de las dos gráficamente.

> **Ejercicio 7:** Implemente una función que calcule la integral de una función $f$ en el intervalo $[a,b]$ usando la fórmula de Simpson compuesta, parametrizando también el número de subintervalos utilizados (se premia la eficiencia). Represente gráficamente la fórmula de Simpson compuesta análogamente a la representación mostrada para la fórmula de los trapecios compuesta.

>**Ejercicio 8:** Realice una comparativa entre los errores cometidos al aproximar funciones potencias sucesivas $x^i$, con $i=1,2,3,\ldots$ en el intervalo $[0,1]$ mediante las fórmulas de los trapecios y de Simpson compuestas con diferentes valores de $h$; e inducir de ahí, los posibles órdenes de exactitud (a partir de los valores de $i$ que proporcionen valores exactos de la integral). No deje de tener en cuenta los posibles errores de redondeo, inevitables en todo cálculo realizado con ordenador.

>**Ejercicio 9:** Programar las técnicas de integración de Romberg y adaptativa, para después aplicarlas a la aproximación de la siguiente integral $$\int_a^b p(x)\, dx$$
>siendo  $\;a=\displaystyle\min_{0\leq i\leq 7}{d_i}$, $\;b=\displaystyle\max_{0\leq i\leq 7}{d_i}$ y
>$$p(x)=d_0 + d_1 x + d_2 x^2 + d_3 x^3+ d_4 x^4 + d_5 x^5 + d_6 x^6 + d_7 x^7 $$
>(siendo $d_0, d_1, \ldots, d_7$ los dígitos de su DNI, pasaporte o tarjeta de residente).

> **Ejercicio 10:** Calcule la misma integral del ejercicio 9 pero utilizando la orden `quad` y compare los resultados de la integración Romberg y adaptativa manuales con los de la orden `quad`.